# Exercise 3 - Putting it all together - Model Training

In this notebook we will tie everything together to and use what we have learnt to try our hand in the competition.

The steps are similar to Exercise 2:
1. Explore the competition dataset
2. Build a convnet from scratch that performs reasonably well
3. Evaluate training and validation accuracy
4. Score the model against test set and submit result

Let's get started!

## Download and Explore the Dataset

Let's start by downloading our dataset, a .zip of 1,226 PNG pictures of different poses, and extracting it locally.

The contents of the `.zip` are extracted to the base directory, which contains `train` and `val` subdirectories for you to do training and validation. The folders have the following structure:

```
---------------
train
|- ChairPose
|- ChildPose
|- Dabbing
|- HandGun
|- HandShake
|- HulkSmash
|- KoreanHeart
|- KungfuCrane
|- KungfuSalute
|- Salute
|- WarriorPose

val
|- ChairPose
|- ChildPose
|- Dabbing
|- HandGun
|- HandShake
|- HulkSmash
|- KoreanHeart
|- KungfuCrane
|- KungfuSalute
|- Salute
|- WarriorPose
```

In [ ]:
# Creating two directories - "data" and "data/trainset_11classes_0_00000" 
!mkdir data && mkdir data/trainset_11classes_0_00000
# Downloading the ai-camp competition dataset
!wget -N https://ai-camp.s3-us-west-2.amazonaws.com/trainset_11classes_0_00000.zip
# Unzip the data into the folder "data/trainset_11classes_0_00000"
!unzip -qq -n trainset_11classes_0_00000.zip -d data/trainset_11classes_0_00000
# Switch directory to "data/trainset_11classes_0_00000" and show its content
!cd data/trainset_11classes_0_00000 && ls

In [ ]:
import os

base_dir = 'data/trainset_11classes_0_00000'

# Directory to our training data
train_folder = os.path.join(base_dir, 'train')

# Directory to our validation data
val_folder = os.path.join(base_dir, 'val')

Now, let's find out the total number of images we have in each `train`, `val` and `test`.

In [ ]:
# List folders and number of files
print("Directory, Number of files")
for root, subdirs, files in os.walk(base_dir):
    print(root, len(files))

We can see that there are 11 categories/folders in each `train` and `val` folder.

Now let's take a look at a few images to get a better sense of what the `KoreanHeart` and `KungfuCrane` categories look like. First, configure the matplotlib parameters:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

Now, display a batch of 8 KoreanHeart and 8 KungfuCrane poses. You can rerun the cell to see a new batch.

In [ ]:
## Path to KoreanHeart and KungfuCrane
train_koreanheart_dir= "data/trainset_11classes_0_00000/train/KoreanHeart"
train_kungfucrane_dir= "data/trainset_11classes_0_00000/train/KungfuCrane"
train_koreanheart_fnames = os.listdir(train_koreanheart_dir)
train_kungfucrane_fnames = os.listdir(train_kungfucrane_dir)

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(15, 15)

pic_index += 8
next_koreanheart_pix = [os.path.join(train_koreanheart_dir, fname) 
                for fname in train_koreanheart_fnames[pic_index-8:pic_index]]
next_kungfucrane_pix = [os.path.join(train_kungfucrane_dir, fname) 
                for fname in train_kungfucrane_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_koreanheart_pix+next_kungfucrane_pix):
    # Set up subplot; subplot indices start at 1
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)
    
    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

## Data Preprocessing

Let's set up data generators that will read images from our source folders and convert them to float32 tensors. We'll have one generator for each training and validation folders.

### Batch
Our generators will yield batches of `32` images of size `299 x 299` and their labels.

### Feature scaling
Recall that in our MNIST/CIFAR-10 exercises, data that goes into a neural network should be normalised in a way that is easier to be processed by the network. In our case, we will preprocess our images by normalising the pixels values to be in the 0 to 1 range. This happens by dividing each pixel value by 255 and this process is known as data normalisation or rescaling.

### Generator - ImageDataGenerator
To rescale the data, we use `keras.preprocessing.image.ImageDataGenerator` class with the `rescale` parameter. This class will also allow us to instantiate generators of augmented image batches (and their labels) via `.flow_from_directory(directory)`. These generators can then be used with the Keras model methods that accept data generators as inputs such as `fit_generator`, `evaluate_generator` and `predict_generator`. We used data augmentation for the training image generator. To find out more about how to do image augmentation in keras, go [here](https://keras.io/preprocessing/image/).

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Batch size
bs = 32

# All images will be resized to this value
image_size = (299, 299)

# All images will be rescaled by 1./255. We apply data augmentation here.
train_datagen = ImageDataGenerator(rescale=1./255,
                                   brightness_range= [0.5,1.5],
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
print("Preparing generator for train dataset")
train_generator = train_datagen.flow_from_directory(
    directory= train_folder, # This is the source directory for training images 
    target_size=image_size, # All images will be resized to value set in image_size
    batch_size=bs,
    class_mode='categorical')

# Flow validation images in batches of 32 using val_datagen generator
print("Preparing generator for validation dataset")
val_generator = val_datagen.flow_from_directory(
    directory= val_folder, 
    target_size=image_size,
    batch_size=bs,
    class_mode='categorical')

## Building a Small Convnet Model

The images that will go into our convnet are **299 x 299** color images. You are free to resize the images for faster training time or better accuracy.

Here, we designed a very simple architecture. It consists of three convolutional layers of stride 2 ending with a [global average pooling layer](https://alexisbcook.github.io/2017/global-average-pooling-layers-for-object-localization/). In this architecture we decided not to use a series of dense layers but instead went straight to the final softmax classification layer. Feel free to modify the model to improve the accuracy. We have introduced here a few new techniques different from the preceeding notebooks but there are many more out there that you can explore and use.

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D, Dense, Dropout, MaxPooling2D

# Here we specify the input shape of our data 
# This should match the size of images ('image_size') along with the number of channels (3)
input_shape = (299, 299, 3)

# Define the number of classes
num_classes = 11

# Defining a baseline model. Here we use the [keras functional api](https://keras.io/getting-started/functional-api-guide) to build the model. 
# TODO: explore different architectures and training schemes
input_img = Input(shape=input_shape)
x = Conv2D(16, (3, 3), padding='same', strides=2)(input_img)
x = Activation('relu')(x)

x = Conv2D(16, (3, 3), padding='same', strides=2)(x)
x = Activation('relu')(x)

x = Conv2D(16, (3, 3), padding='same', strides=2)(x)
x = Activation('relu')(x)

x = Conv2D(16, (3, 3), padding='same', strides=2)(x)
x = Activation('relu')(x)

x = GlobalAveragePooling2D()(x)

predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=input_img, outputs=predictions)

Let's summarise the model architecture:

In [ ]:
model.summary()

Next, we will configure the specifications for model training.

We train our model with `categorical_crossentropy` loss, because this is a multi-class problem. We will use the `adam` optimizer with default settings. During training, we want to monitor `accuracy` of the classification.

In [ ]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

## Setting Up Checkpoints

Let's setup a [checkpoint](https://keras.io/callbacks/) to help us monitor the validation accuracy as the model trains. This checkpoint will save the model with best validation accuracy seen so far.

In [ ]:
from keras.callbacks import ModelCheckpoint

bestValidationCheckpointer = ModelCheckpoint('saved_model.hdf5', monitor='val_acc', save_best_only=True, verbose=1)

## Model Training 

Let's train on all the images in the training set, for 100 epochs, and validate against all validation images.

Note: This may take a while to run.

In [ ]:
history = model.fit_generator(
        train_generator, # train generator has 973 train images
        steps_per_epoch=train_generator.samples // bs + 1,
        epochs=200,
        validation_data=val_generator, # validation generator has 253 validation images
        validation_steps=val_generator.samples // bs + 1,
        callbacks=[bestValidationCheckpointer]
)

## Evaluating Accuracy and Loss of the Model

With a trained model, we can evaluate the model performance against the truth labels of our validation set. First, we load the best model encountered during training.

In [ ]:
from keras.models import load_model

model_path = 'saved_model.hdf5'
model = load_model( model_path )

Then, we validate accuracy of the loaded model on our good old validation set.

In [ ]:
val_generator.reset()

scores = model.evaluate_generator(val_generator, steps=val_generator.samples // val_generator.batch_size + 1, verbose=1)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])